# Installating Scilab on Jupyter-Portable (www.portabledevapps.net) via a Notebook

In [ ]:
import os
import sys
import io

In [ ]:
try:
    import urllib.request as urllib2  # Python 3
except:
    import urllib2  # Python 2
if 'amd64' in sys.version.lower():
    scilab_exe="scilab-6.0.1_x64.exe"
    scilab_url="http://www.scilab.org/download/6.0.1/scilab-6.0.1_x64.exe"
    hashes="57c915430bd4a47b0c0bdc124a96a2190fa527ac46978f407f2d1689f3b566f7"
else:
    scilab_exe="scilab-6.0.1.exe"
    scilab_url="http://www.scilab.org/download/6.0.1/scilab-6.0.1.exe"
    hashes="425e08b41eb5b5bdad0599234a5c4c430caf86c333af8d5e61f16f4b6d38b265"
    
scilab_installer=os.environ["APPSDIR"]+"\\datafiles\\"+scilab_exe

In [ ]:
# start download
g = urllib2.urlopen(scilab_url) 
with io.open(scilab_installer, 'wb') as f:
    f.write(g.read())
g.close
g = None

In [ ]:
# checking the hashes
import hashlib
def give_hash(of_file, with_this):
    with io.open(scilab_installer, 'rb') as f:
        return with_this(f.read()).hexdigest()  
print (" "*30+"SHA-256"+" "*30+"\n"+"-"*65)

print ("%s %s" % (give_hash(scilab_installer, hashlib.sha256),scilab_installer))
assert give_hash(scilab_installer, hashlib.sha256) == hashes

In [ ]:
# create setup configuration INF file
inf_txt = r"""[Setup]
Lang=en_EN
Dir=
DisableProgramGroupPage=yes
Group=
NoIcons=1
SetupType=full
Components=scilab,jvm_module,jvm_module\graphics,jvm_module\graphics\xcos,jvm_module\scinotes,jvm_module\jre_1_6,cpu_optimization,cpu_optimization\mkl,fftw,fftw\ref_library,development_tools,development_tools\modules_manager,development_tools\toolbox_skeleton,development_tools\tests,mpi,tclsci
Tasks=""
"""

inf_txt = inf_txt.replace("Dir=","Dir="+os.environ["APPSDIR"]+"\\scilab")

if sys.version_info[0] == 3:
    with io.open("scilab_install.inf", 'w', encoding = sys.getdefaultencoding() ) as f:
        for line in inf_txt.splitlines():
            f.write('%s\n' %  line  )
else:
    with io.open("scilab_install.inf", 'wb'  ) as f:
        for line in inf_txt.splitlines():
            f.write('%s\r\n' %  line.encode(sys.getdefaultencoding()) )

In [ ]:
# install scilab
!start cmd /C %scilab_installer% /SP- /VERYSILENT /SUPPRESSMSGBOXES /NORESTART /NOICONS /LOADINF=scilab_install.inf

In [ ]:
# install scilab Jupyter kernel
!start cmd /C pip install scilab_kernel

In [ ]:
SCILAB_EXECUTABLE

# add scilab environment variables to the Jupyter Environment
inp_str = r"""
rem ******************
rem handle scilab if included
rem ******************
if not exist "%APPSDIR%\scilab" goto scilab_bad
set SCILAB_EXECUTABLE=%APPSDIR%\octave\mingw64\bin\octave-cli.exe
:scilab_bad
"""

if not ('amd64' in sys.version.lower()):
    inp_str = inp_str.replace('mingw64','mingw32')

# the env.bat has the "hidden" file attribute and is therefore not writable
os.system("attrib -h "+os.environ["APPSDIR"]+"\\scripts\\env.bat")

with open(os.environ["APPSDIR"]+"\\scripts\\env.bat", "a") as myfile:
    myfile.write(inp_str)
    
# restore the hidden file attribute of env.bat
os.system("attrib +h "+os.environ["APPSDIR"]+"\\scripts\\env.bat")

In [ ]:
# clean up
os.remove("scilab_install.inf")